In [1]:
from ultralytics import SAM, YOLO
from ultralytics.models.sam import SAM3SemanticPredictor
import os
import cv2
import torch
import numpy as np


In [ ]:
!wget https://www.modelscope.cn/models/facebook/sam3/resolve/master/sam3.pt

--2025-12-21 10:50:09--  https://www.modelscope.cn/models/facebook/sam3/resolve/master/sam3.pt
Resolving www.modelscope.cn (www.modelscope.cn)... 47.251.62.57
Connecting to www.modelscope.cn (www.modelscope.cn)|47.251.62.57|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-cn-1.modelscope.cn/prod/lfs-objects/99/99/e2341ceef5e136daa386eecb55cb414446a00ac2b55eb2dfd2f7c3cf8c9e?filename=sam3.pt&namespace=facebook&repository=sam3&revision=master&tag=model&auth_key=1766314211-bd74c36d11c84fe4af7f97236fb5195c-0-18ef8795d8ac64f9bfd6e033f617f7df [following]
--2025-12-21 10:50:11--  https://cdn-lfs-cn-1.modelscope.cn/prod/lfs-objects/99/99/e2341ceef5e136daa386eecb55cb414446a00ac2b55eb2dfd2f7c3cf8c9e?filename=sam3.pt&namespace=facebook&repository=sam3&revision=master&tag=model&auth_key=1766314211-bd74c36d11c84fe4af7f97236fb5195c-0-18ef8795d8ac64f9bfd6e033f617f7df
Resolving cdn-lfs-cn-1.modelscope.cn (cdn-lfs-cn-1.modelscope.cn)... 163.181.22.188, 163.1

In [2]:
img = cv2.imread('assets/bus.jpg')
img.shape

(1080, 810, 3)

In [3]:
def clean_overlapping_masks(result):
    """
    Hàm lọc bỏ các pixel chồng lấn dựa trên confidence score.
    Input: result object từ YOLO model.predict()
    Output: Tensor masks mới (N, H, W) không còn pixel chồng nhau.
    """
    if result.masks is None:
        return None

    # 1. Lấy dữ liệu masks và confidence
    # masks.data có dạng (N, H, W) - N là số object, H, W là kích thước mask
    masks = result.masks.data 
    confs = result.boxes.conf  # Tensor (N,) chứa điểm confidence của từng box
    
    # Lấy kích thước
    N, H, W = masks.shape
    
    if N == 0:
        return masks

    # 2. Tạo bản đồ trọng số (Weighted Map)
    # Reshape confs thành (N, 1, 1) để nhân với masks (Broadcasting)
    # Kết quả: Pixel nào thuộc mask sẽ có giá trị = confidence, không thì = 0
    confs_reshaped = confs.view(N, 1, 1)
    weighted_masks = masks * confs_reshaped # (N, H, W)

    # 3. Tìm chỉ số của mask có điểm cao nhất tại mỗi pixel
    # max_vals: Giá trị conf lớn nhất tại pixel đó
    # max_indices: Chỉ số (index) của mask sở hữu giá trị lớn nhất đó
    max_vals, max_indices = torch.max(weighted_masks, dim=0) # (H, W)

    # 4. Tái tạo lại masks không chồng lấn (Clean Masks)
    # Tạo một tensor chứa các index [0, 1, ..., N-1] để so sánh
    idx_range = torch.arange(N, device=masks.device).view(N, 1, 1)
    
    # Logic: Một pixel thuộc về mask i KHI VÀ CHỈ KHI:
    # - Mask i là mask chiến thắng tại pixel đó (max_indices == i)
    # - Và pixel đó thực sự có dữ liệu (masks[i] > 0) - để tránh lấy background
    clean_masks = (max_indices.unsqueeze(0) == idx_range) & (masks > 0)

    # Chuyển về dạng float giống mask gốc
    return clean_masks.float(), clean_masks.float().sum(dim=0)


In [4]:
overrides = dict(
    conf=0.5,
    task="segment",
    mode="predict",
    model="sam3.pt",
    half=True,  # Use FP16 for faster inference
    save=True,
    overlap_mask=False
)

model = SAM3SemanticPredictor(overrides=overrides) 

In [5]:
results = model('assets/bus.jpg', text=["person", "bus", "glasses"])
# results = model('assets/bus.jpg', text=["bus"])


Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3090, 24152MiB)


/opt/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


WARNING ⚠️ imgsz=[640] must be multiple of max stride 14, updating to [644]
image 1/1 /workspace/VLM2Vec/assets/bus.jpg: 644x644 5 persons, 1 bus, 2 glassess, 386.4ms
Speed: 8.3ms preprocess, 386.4ms inference, 3.5ms postprocess per image at shape (1, 3, 644, 644)
Results saved to /workspace/VLM2Vec/runs/segment/predict2


In [ ]:
res = results[0]
print(f"Before cleaning: {res.masks.data.shape} masks")
clean_masks, agg_clean_masks = clean_overlapping_masks(res)
res.data = clean_masks  # Cập nhật lại dữ liệu mask đã lọc
# res.save()

Before cleaning: torch.Size([8, 1080, 810]) masks


'results_bus.jpg'

In [7]:
clean_masks = res.data
clean_masks.sum(dim=0).max()

tensor(1., device='cuda:0')

In [9]:
agg_clean_masks.shape
torch.save(agg_clean_masks, 'assets/bus_mask.pt')